# LPC from scratch

This implementation is to study how to implement LPC from scratch. 
After that, we will compare the results with the LPC implementation from librosa.

### Questions
- What are differences between the two methods: (1) Yule-walker's and (2) Burg's methods?
    - Yule-walker's method: 
        - Autocorrelation
        - Minimize residual energy
        - Apply Levinson-Durbin recursion to find LPC coefficients
    - Burg's method:
        - A consrtainted least-squares problem using the sum of forward and backward prediction errors energies.


### Steps:
- Define prediction error
    - Residual energy
- Autocorrelation
- Minimize residual energy
- Apply Levinson-Durbin recursion to find LPC coefficients

In [2]:
import numpy as np
import pandas as pd
import librosa

#Check the Yule-Walker equation (e.g. Obtain R_l and trace the Perform Levinson-Durbin algorithm, etc.)

In [3]:
# load data
df_data = pd.read_csv('../data_path.csv')
df_data.head()

,Unnamed: 0,id,audio_path,process,pass_number,tooth_number,label
0,0,10,/net/papilio/storage4/database/jtekt/20230216/...,rough,1,1,0
1,1,10,/net/papilio/storage4/database/jtekt/20230216/...,rough,2,2,0
2,2,10,/net/papilio/storage4/database/jtekt/20230216/...,rough,3,3,0
3,3,10,/net/papilio/storage4/database/jtekt/20230216/...,rough,4,1,0
4,4,10,/net/papilio/storage4/database/jtekt/20230216/...,finishing,1,1,0


In [4]:
swav, sr = librosa.load(df_data['audio_path'].values[0], sr=22050)
print(f'wave shape: {swav.shape}')
print(f'with sampling rate: {sr}')

wave shape: (72765,)
with sampling rate: 22050


In [14]:
# Burg's method
def burg_method(x, order):
    """ implementation of Burg's method for calculating AR coefficients """
    print(f'x shape: {x.shape}')
    # initialize variables
    dtype = x.dtype
    print(f'dtype: {dtype}')
    shape = list(x.shape)
    shape[0] = order + 1
    print(f'shape=order+1: {shape}')

    ar_coeffs = np.zeros(tuple(shape), dtype=dtype)
    print(f'ar_coeffs: {ar_coeffs}')
    ar_coeffs[0] = 1
    print(f'ar_coeffs[0]: {ar_coeffs}')
    ar_coeffs_prev = ar_coeffs.copy()
    print(f'ar_coeffs_prev: {ar_coeffs_prev}')

    shape[0] = 1
    print(f'shape: {shape}')
    reflect_coeff = np.zeros(shape, dtype=dtype) # a reflection coefficient is ..
    print(f'reflect_coeff: {reflect_coeff}')
    den = reflect_coeff.copy()
    print(f'den: {den}')

    epsilon = np.finfo(dtype).tiny # epsilon is the smallest representable number such that 1.0 + epsilon != 1.0
    print(f'epsilon: {epsilon}')

    # Initialize the forward and backward prediction errors
    fwd_pred_error = x[1:]
    bwd_pred_error = x[:-1]
    print(f'fwd_pred_error shpe: {fwd_pred_error.shape}')
    print(f'bwd_pred_error shape: {bwd_pred_error.shape}')

    den[0] = np.sum(fwd_pred_error**2 + bwd_pred_error**2, axis=0) #eq16
    print(f'den: {den}')

    for i in range(order): # has to be range of order
        print(f'============ iteration of order {i} =================')
        # eq15
        print(f'backward error: {bwd_pred_error}, sum: {np.sum(bwd_pred_error)}')
        print(f'forward error: {fwd_pred_error}, sum: {np.sum(fwd_pred_error)}')
        print(f'Combine together: {np.sum(bwd_pred_error) + np.sum(fwd_pred_error)}') 
        reflect_coeff[0] = np.sum(bwd_pred_error + fwd_pred_error, axis=0)
        print(f'reflect_coeff: {reflect_coeff}')
        reflect_coeff[0] *= -2
        print(f'reflect_coeff: {reflect_coeff}')
        reflect_coeff[0] /= den[0] + epsilon
        print(f'reflect_coeff: {reflect_coeff}')

        # compute all AR coefficients
        print(f'----- before swapping -----')
        print(f'ar_coeffs: {ar_coeffs}')
        print(f'ar_coeffs_prev: {ar_coeffs_prev}')
        ar_coeffs_prev, ar_coeffs = ar_coeffs, ar_coeffs_prev # flip them each time we increase the model order so that we may use all the coefficients from the previous order while we compute those for the new one.
        print(f'----- after swapping -----')
        print(f'ar_coeffs: {ar_coeffs}')
        print(f'ar_coeffs_prev: {ar_coeffs_prev}')

        # eq5: the recursive Levinson-Durbin algorithm
        for j in range(1, i + 2): # looping start from 1 to nth order+1
            # reflection multiply should be broadcast
            print(f'i+1-j: {i+1-j}')
            print(f'ar_coeffs_prev[j]: {ar_coeffs_prev[j]}, reflect_coeff[0]: {reflect_coeff[0]}, ar_coeffs_prev[i + 1 - j]: {ar_coeffs_prev[i + 1 - j]}')
            ar_coeffs[j] = (
                ar_coeffs_prev[j] + reflect_coeff[0] * ar_coeffs_prev[i + 1 - j]
            )
            print(f'ar_coeffs: {ar_coeffs}')
        
        # update forward and backward prediction errors as eq13 and eq14
        fwd_pred_error_tmp = fwd_pred_error
        fwd_pred_error = fwd_pred_error_tmp + reflect_coeff * bwd_pred_error
        bwd_pred_error = bwd_pred_error + reflect_coeff * fwd_pred_error_tmp

        # compute denominator using recursion from eqn 17.
        q = 1.0 - reflect_coeff[0] ** 2
        print(f'q: {q}')
        den[0] = q * den[0] - bwd_pred_error[-1] ** 2 - fwd_pred_error[-1] ** 2
        print(f'den: {den}')
        
        # shiftup forward error
        fwd_pred_error = fwd_pred_error[1:]
        bwd_pred_error = bwd_pred_error[:-1]
        print(f'------- after shifting ------- ')
        print(f'fwd_pred_error: {fwd_pred_error}')
        print(f'bwd_pred_error: {bwd_pred_error}')
    return ar_coeffs

x = swav.copy()
order = 5
print(f'AR coeff: {burg_method(x, order)}')

x shape: (72765,)
dtype: float32
shape=order+1: [6]
ar_coeffs: [0. 0. 0. 0. 0. 0.]
ar_coeffs[0]: [1. 0. 0. 0. 0. 0.]
ar_coeffs_prev: [1. 0. 0. 0. 0. 0.]
shape: [1]
reflect_coeff: [0.]
den: [0.]
epsilon: 1.1754943508222875e-38
fwd_pred_error shpe: (72764,)
bwd_pred_error shape: (72764,)
den: [6447.974]
============ iteration of order 0 =================
backward error: [-0.33111086 -0.09778264  0.31707639 ...  0.18342648 -0.05202613
 -0.10739426], sum: -363.2904357910156
forward error: [-0.09778264  0.31707639 -0.22779685 ... -0.05202613 -0.10739426
  0.30542496], sum: -362.65386962890625
Combine together: -725.9443359375
reflect_coeff: [-725.94434]
reflect_coeff: [1451.8887]
reflect_coeff: [0.22516975]
----- before swapping -----
ar_coeffs: [1. 0. 0. 0. 0. 0.]
ar_coeffs_prev: [1. 0. 0. 0. 0. 0.]
----- after swapping -----
ar_coeffs: [1. 0. 0. 0. 0. 0.]
ar_coeffs_prev: [1. 0. 0. 0. 0. 0.]
i+1-j: 0
ar_coeffs_prev[j]: 0.0, reflect_coeff[0]: 0.22516974806785583, ar_coeffs_prev[i + 1 - j]: 

In [16]:
librosa.lpc(x, order=order, axis=0)

array([ 1.        ,  0.23022743,  0.6526499 , -0.17364112,  0.18484306,
       -0.08656871], dtype=float32)

In [17]:
x = swav.copy()
order = 5
# print with type
print(librosa.lpc(x, order=order, axis=0))

[ 1.          0.23022743  0.6526499  -0.17364112  0.18484306 -0.08656871]


# Defining error measure

In [10]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



# Minimizing error